# Low resource language model training and evaluation

## Required packages

In [1]:
from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline
import matplotlib.pylab as plt

In [3]:
from glob import glob
import json, sys, os, librosa
import numpy as np
import pandas as pd
from jiwer import wer
from unidecode import unidecode
from tqdm.notebook import tqdm

In [4]:
sys.path.append('/home/catskills/Desktop/openasr20/end2end_asr_pytorch')
os.environ['IN_JUPYTER']='True'
os.environ['CUDA_VISIBLE_DEVICES']='1'

## Configuration

In [9]:
from config import C

In [13]:
C.language='pashto'
C.batch_size=12
C.update()

## Initial BUILD setup

In [ ]:
from transcript_to_split_BUILD_wavs import transcript_to_split_BUILD_wavs
if not os.path.exists(f'{C.stage}/openasr20_{C.language}/build/audio_split'):
    transcript_to_split_BUILD_wavs(C)

 70%|███████   | 92/131 [00:26<00:13,  2.92it/s]

In [16]:
srcs = glob(f'{C.stage}/openasr20_{C.language}/build/transcription/*.txt')
tasks = [(x,x.replace('.txt','.stm').replace('/transcription/','/transcription_stm/')) for x in srcs]
for src,tgt in tqdm(tasks):
    if not os.path.exists(tgt):
        os.system(f"python OpenASR_convert_reference_transcript.py -f {src} -o {C.stm_dir}")

In [17]:
from transcript_to_grapheme_dictionary import transcript_to_grapheme_dictionary
if not os.path.isfile(C.grapheme_dictionary_fn):
    transcript_to_grapheme_dictionary(C)

## BUILD advanced sample treatment: SD, NR, norm, trim

In [18]:
from advanced_sample_treatment_for_BUILD import advanced_sample_treatment_for_BUILD
advanced_sample_treatment_for_BUILD(C)

  0%|          | 0/8665 [00:00<?, ?it/s]


ValueError: could not convert string to float: 'inLine'

## Training regime: first 10% audio length, first 20%....first 100%

In [15]:
os.system("/bin/rm -rf runs")
os.system("python runtrainer.py")

## Afterburner training

## Build 4K inference

## Build afterburner inference

## Build model validation

In [ ]:
os.system(f"""python end2end_asr_pytorch/infer.py \
	--continue-from save/{C.language}_end2end_asr_pytorch_drop0.1_cnn_batch12_4_vgg_layer4/best_model.th \
	--test-manifest-list manifest.csv \
	--batch-size {C.batch_size} --output analysis/{C.language}/TEST_BUILD.txt --cuda --verbose""")

## SCLite scoring

In [ ]:
C.translation_file="{C.recording} analysis/{C.language}/TEST_BUILD.txt"
package_BUILD_as_stm(C)

In [ ]:
C.ref_file="""analysis/{C.language}/build_reference/{recording}.stm"""
C.hyp_file="""analysis/{C.language}/build_inference/{C.recording}.ctm"""
C.scoring="""scoring"""
os.system("""mkdir -p scoring""")

In [ ]:
cmd="""SCTK/bin/sclite -r {C.ref_file} stm -h {C.hype_file} ctm -F -D -O {C.scoring} -o sum rsum pralign prf -e utf-8"""

## DEV basic preparation

## DEV advanced preparation: SD, NR, norm, 4K, SD, NR, norm, max split, trim

## DEV 4K infer

## DEV Afterburner infer

## NIST model test

In [ ]:
from split_DEV_audio import split_DEV_audio
split_DEV_audio(C)

In [ ]:
from make_DEV_infer_csv import make_DEV_infer_csv
make_DEV_infer_csv(C)

In [ ]:
os.system(f"""python end2end_asr_pytorch/infer.py \
	--continue-from save/{C.language}_end2end_asr_pytorch_drop0.1_cnn_batch12_4_vgg_layer4/best_model.th \
	--test-manifest-list analysis/{C.language}/DEV_{C.language}_split.csv \
	--batch-size {C.batch_size} \
	--output analysis/{C.language}/RESULT_{C.language}.txt \
	--cuda \
	--verbose""")

In [ ]:
from trim_repeats import trim_repeats
trim_repeats(C,cfn,tfn)

In [ ]:
from package_DEV import package_DEV
package_DEV(C)